# Build custom databases with segmented flu genomes using kraken_flu utility
A new utilty [kraken_flu](https://gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu) was created to generate Kraken2 databases with segmented flu genomes from any set of input files (either from kraken2 or directly from NCBI).

The tool identifies influenza A H1N1, A H3N2 and B in the genome fasta file and creates new artificial taxa in the taxonomy files of the database build. Each segment of any of the viruses of interest is assign the original whole genome as the parent and will be listed as a new taxon in the Kraken2 output files.

# install the kraken_flu tool
The tool can be installed from local gitlab. Creating a venv for it.

In [27]:
python3 -m venv ~/kraken_flu

  Cloning ssh://****@gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu.git to /tmp/pip-install-1czikc84/kraken-flu_e04ea4d0e306496fadd281fe42d9687d
  Running command git clone --filter=blob:none --quiet 'ssh://****@gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu.git' /tmp/pip-install-1czikc84/kraken-flu_e04ea4d0e306496fadd281fe42d9687d
  Resolved ssh://****@gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu.git to commit 277e12048c3b39c50d23bcc55f7573051872efc3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
~/kraken_flu/bin/pip install kraken_flu@git+ssh://git@gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu.git

  Cloning ssh://****@gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu.git to /tmp/pip-install-kq0vv75v/kraken-flu_b460602e5d5b43698d54c603f3f7e0bf
  Running command git clone --filter=blob:none --quiet 'ssh://****@gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu.git' /tmp/pip-install-kq0vv75v/kraken-flu_b460602e5d5b43698d54c603f3f7e0bf
  Resolved ssh://****@gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu.git to commit 6995f865433d45b51aa5978dec0671ddb2b2b0d6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kraken_flu: filename=kraken_flu-1.1.1.dev1+g6995f86-py2.py3-none-any.whl size=17847 sha256=766287122377d3be0e94005ea0b223e1f9d91ab947a642e1a0288600c722c8b5
  Stored in directory: /tmp/pip-ephem-wheel-cache-mw1t3aqk/wheels/36/3d/a4/e69d3d0a667ae4fbdd235a419321c7e4986b2c2df02ecdb5de
Successfully built kraken_flu
  Attempting uninstall: kra

In [2]:
~/kraken_flu/bin/kraken_flu -v

kraken_flu 1.1.1.dev1+g6995f86


## Download common data
The following data will be used by several subsequent database builds. The individual databases will be using copies of the relevant files in these download direcotries so we do not have to keep re-downloading the large files involved.

### Download NCBI taxonomy data
Use the kraken2 build tool to download taxonomy files from [NCBI](https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/). These files will be needed for all database builds.

In [2]:
module load kraken2/2.1.2

        Module loaded. For more information run 'module help kraken2/2.1.2'.
        - By default, kraken2 will search for named databases in /data/pam/software/kraken2 and the current working directory. The full path to
        any database can also be used.
        - `kraken2-build` commands will sometimes experience an error suggesting that it is not possible to download a library using rsync. While the option --use-ftp
        may fix this, rsync tends to be faster and more reliable. Try re-running the `kraken2-build` command until the download is successful. Note that `kraken2-build`
        will produce checkpoints during the installation process and will restart at the last successful step.
        - If `kraken2-build build` freezes or hangs indefinitely, using the `--fast-build` option may help. An earlier version of the software (if available) may also
        allow the build step to proceed successfully.
>


In [3]:
export BASE_DIR=/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu/

In [7]:
mkdir -p ${BASE_DIR}

In [4]:
export TAX_PATH=${BASE_DIR}/taxonomy_download/

In [5]:
export LIB_PATH_VIR=${BASE_DIR}/library_download_viral/

In [10]:
mkdir -p {$TAX_PATH}
mkdir -p ${LIB_PATH_VIR}

In [11]:
kraken2-build --download-taxonomy --db ${TAX_PATH}

Downloaded accession to taxon map(s)
Uncompressing taxonomy data... done.
Untarring taxonomy tree data... done.


<a id="downloadrefseq"></a>
### Download viral RefSeq
This kraken2 build command downloads RefSeq and assigns taxonomy to the sequences to create a ready dataset for DB creation.

___NOTE___: for some reason, the rsync command often fails and it is very slow (seems to download sequences one by one?), so it makes sense to keep this directory as a resource for further databases. RefSeq is not updated very often so it should not be a problem with being up to date

In [12]:
kraken2-build \
    --download-library viral \
    --db ${LIB_PATH_VIR}

Step 1/2: Performing rsync file transfer of requested files
Rsync file transfer complete.
Step 2/2: Assigning taxonomic IDs to sequences
Processed 14972 projects (18639 sequences, 549.88 Mbp)... done.
All files processed, cleaning up extra sequence files... done, library complete.
Masking low-complexity regions of downloaded library... done.


___Resulting files:___
These are the files created by the downloads. The file library.fna is the genome sequence file. As this was downloaded from a kraken2 pre-built repository of refseq, the FASTA headers already contain taxonomy IDs, which will be modified by the kranek_flu tool.


In [7]:
tree ${BASE_DIR}

/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu/
├── db1-allrefseq_wo_human
│   ├── library
│   │   └── library.fna
│   └── taxonomy
│       ├── modes.dmp
│       └── names.dmp
├── library_download_viral
│   └── library
│       └── viral
│           ├── assembly_summary.txt
│           ├── library.fna
│           ├── library.fna.masked
│           ├── manifest.txt
│           └── prelim_map.txt
└── taxonomy_download
    └── taxonomy
        ├── accmap.dlflag
        ├── citations.dmp
        ├── delnodes.dmp
        ├── division.dmp
        ├── gc.prt
        ├── gencode.dmp
        ├── images.dmp
        ├── merged.dmp
        ├── names.dmp
        ├── nodes.dmp
        ├── nucl_gb.accession2taxid
        ├── nucl_wgs.accession2taxid
        ├── readme.txt
        ├── taxdump.dlflag
        ├── taxdump.tar.gz
        └── taxdump.untarflag

8 directories, 24 files


# Use kraken_flu to create a database with all of viral refseq
The tool will convert the taxonomy and library files and write the modified copies to a new directory. The modified files will have new taxon IDs in the FASTA header for all flu A H1N1, A H3N2 and B viral genome sequences. The pattern is currently hardcoded into the tool but can be easily extended to cover other viruses.

In [60]:
# FORCE DELETE any existing direcotry
rm -rf ${BASE_DIR}/db1-allrefseq_wo_human

# run the utility
~/kraken_flu/bin/kraken_flu build \
    --library_path ${LIB_PATH_VIR}/library/viral/ \
    --taxonomy_path ${TAX_PATH}/taxonomy \
    --out_dir ${BASE_DIR}/db1-allrefseq_wo_human

2023-12-18 16:52:53,797 writing modified FASTA file to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//db1-allrefseq_wo_human/library/library.fna
2023-12-18 16:52:53,812 scanning file /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//library_download_viral//library/viral/library.fna for influenza viruses
2023-12-18 16:52:57,121 done - found 32 segment sequences in FASTA file
2023-12-18 16:53:03,534 finished writing modified FASTA file
2023-12-18 16:53:03,537 writing modified prelim_map.txt file to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//db1-allrefseq_wo_human/library/prelim_map.txt
2023-12-18 16:53:03,594 finished writing modified prelim_map.txt file
2023-12-18 16:53:04,298 writing modified names file to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//db1-allrefseq_wo_human/taxonomy/names.dmp
2023-12-18 16:53:04,495 finished writing modified names file
2023-12-18 16:53:05,086 w

In [61]:
tree ${BASE_DIR}/db1-allrefseq_wo_human

/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//db1-allrefseq_wo_human
├── library
│   ├── library.fna
│   └── prelim_map.txt
└── taxonomy
    ├── names.dmp
    └── nodes.dmp

2 directories, 4 files


show the tail of the new names file which now contains segments as new names

In [49]:
tail ${BASE_DIR}/db1-allrefseq_wo_human/taxonomy/names.dmp

3108293	|	NC_002016.1 Influenza A virus (A/Puerto Rico/8/1934(H1N1)) segment 7, complete sequence	|		|	scientific name	|
3108294	|	NC_002020.1 Influenza A virus (A/Puerto Rico/8/1934(H1N1)) segment 8, complete sequence	|		|	scientific name	|
3108295	|	NC_026438.1 Influenza A virus (A/California/07/2009(H1N1)) segment 1 polymerase PB2 (PB2) gene, complete cds	|		|	scientific name	|
3108296	|	NC_026435.1 Influenza A virus (A/California/07/2009(H1N1)) segment 2 polymerase PB1 (PB1) gene, complete cds; and nonfunctional PB1-F2 protein (PB1-F2) gene, complete sequence	|		|	scientific name	|
3108297	|	NC_026437.1 Influenza A virus (A/California/07/2009(H1N1)) segment 3 polymerase PA (PA) gene, complete cds	|		|	scientific name	|
3108298	|	NC_026433.1 Influenza A virus (A/California/07/2009(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds	|		|	scientific name	|
3108299	|	NC_026436.1 Influenza A virus (A/California/07/2009(H1N1)) segment 5 nucleocapsid protein (NP) gene, complete cds	|		

tail of the nodes file shows the new nodes having been added

In [50]:
tail ${BASE_DIR}/db1-allrefseq_wo_human/taxonomy/nodes.dmp

3108293	|	211044	|	no rank	|		|	9	|	1	|	1	|	1	|	0	|	1	|	1	|		|		|
3108294	|	211044	|	no rank	|		|	9	|	1	|	1	|	1	|	0	|	1	|	1	|		|		|
3108295	|	641809	|	no rank	|		|	9	|	1	|	1	|	1	|	0	|	1	|	1	|		|		|
3108296	|	641809	|	no rank	|		|	9	|	1	|	1	|	1	|	0	|	1	|	1	|		|		|
3108297	|	641809	|	no rank	|		|	9	|	1	|	1	|	1	|	0	|	1	|	1	|		|		|
3108298	|	641809	|	no rank	|		|	9	|	1	|	1	|	1	|	0	|	1	|	1	|		|		|
3108299	|	641809	|	no rank	|		|	9	|	1	|	1	|	1	|	0	|	1	|	1	|		|		|
3108300	|	641809	|	no rank	|		|	9	|	1	|	1	|	1	|	0	|	1	|	1	|		|		|
3108301	|	641809	|	no rank	|		|	9	|	1	|	1	|	1	|	0	|	1	|	1	|		|		|
3108302	|	641809	|	no rank	|		|	9	|	1	|	1	|	1	|	0	|	1	|	1	|		|		|


show the modified FASTA headers for the H1N1 sequences

In [63]:
grep H1N1 ${BASE_DIR}/db1-allrefseq_wo_human/library/library.fna

>kraken:taxid|3108287|NC_002023.1 Influenza A virus (A/Puerto Rico/8/1934(H1N1)) segment 1, complete sequence
>kraken:taxid|3108288|NC_002021.1 Influenza A virus (A/Puerto Rico/8/1934(H1N1)) segment 2, complete sequence
>kraken:taxid|3108289|NC_002022.1 Influenza A virus (A/Puerto Rico/8/1934(H1N1)) segment 3, complete sequence
>kraken:taxid|3108290|NC_002017.1 Influenza A virus (A/Puerto Rico/8/1934(H1N1)) segment 4, complete sequence
>kraken:taxid|3108291|NC_002019.1 Influenza A virus (A/Puerto Rico/8/1934(H1N1)) segment 5, complete sequence
>kraken:taxid|3108292|NC_002018.1 Influenza A virus (A/Puerto Rico/8/1934(H1N1)) segment 6, complete sequence
>kraken:taxid|3108293|NC_002016.1 Influenza A virus (A/Puerto Rico/8/1934(H1N1)) segment 7, complete sequence
>kraken:taxid|3108294|NC_002020.1 Influenza A virus (A/Puerto Rico/8/1934(H1N1)) segment 8, complete sequence
>kraken:taxid|3108295|NC_026438.1 Influenza A virus (A/California/07/2009(H1N1)) segment 1 polymerase PB2 (PB2) gene, co

## Build the new database
Use kraken2 build tool to build the final new database from the modified genome and taxonomy files


In [64]:
kraken2-build \
    --build \
    --db ${BASE_DIR}/db1-allrefseq_wo_human

Creating sequence ID to taxonomy ID map (step 1)...
Sequence ID to taxonomy ID map complete. [0.059s]
Estimating required capacity (step 2)...
Estimated hash table requirement: 648810788 bytes
Capacity estimation complete. [20.288s]
Building database files (step 3)...
Taxonomy parsed and converted.
CHT created with 15 bits reserved for taxid.
Completed processing of 18639 sequences, 549880780 bp
Writing data to disk...  complete.
Database files completed. 31.032s]
Database construction complete. [Total: 2m51.403s]


## Test the database
Using existing simulated reads data to test the new database

In [6]:
export TEST_BASE_DIR=/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu_tests/

In [65]:
rm -rf ${TEST_BASE_DIR}
mkdir -p ${TEST_BASE_DIR}

In [66]:
kraken2 \
    --db ${BASE_DIR}/db1-allrefseq_wo_human \
    --output ${TEST_BASE_DIR}/output.kraken \
    --paired \
    --classified-out ${TEST_BASE_DIR}/class_seqs#.fq \
    --unclassified-out ${TEST_BASE_DIR}/unclass_seqs#.fq \
    --report ${TEST_BASE_DIR}/report.txt \
/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/sim_reads/art/mix/mix_of_all_01-1.fq \
/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/sim_reads/art/mix/mix_of_all_01-2.fq


Loading database information... done.
6620 sequences (1.99 Mbp) processed in 0.366s (1085.5 Kseq/m, 325.65 Mbp/m).
  6618 sequences classified (99.97%)
  2 sequences unclassified (0.03%)


In [67]:
cat ${TEST_BASE_DIR}/report.txt

  0.03	2	2	U	0	unclassified
 99.97	6618	0	R	1	root
 99.97	6618	0	D	10239	  Viruses
 99.97	6618	0	D1	2559587	    Riboviria
 99.97	6618	0	K	2732396	      Orthornavirae
 69.91	4628	0	P	2497569	        Negarnaviricota
 39.43	2610	0	P1	2497571	          Polyploviricotina
 39.43	2610	0	C	2497577	            Insthoviricetes
 39.43	2610	0	O	2499411	              Articulavirales
 39.43	2610	0	F	11308	                Orthomyxoviridae
 25.53	1690	0	G	197911	                  Alphainfluenzavirus
 25.53	1690	0	S	2955291	                    Alphainfluenzavirus influenzae
 25.53	1690	0	S1	11320	                      Influenza A virus
 12.84	850	0	S2	119210	                        H3N2 subtype
 12.84	850	0	S3	335341	                          Influenza A virus (A/New York/392/2004(H3N2))
  2.27	150	150	S4	3108271	                            NC_007373.1 Influenza A virus (A/New York/392/2004(H3N2)) segment 1, complete sequence
  2.27	150	150	S4	3108272	                            NC_007372.1 Influenza A

## Conclusion
The kraken-flu tool is working as intended and the output shows the expected assignement of reads at segment level.  
In this database, two references are used for H1N1 and H3N2, respectively. The outcome still assigns the reads to the correct reference from which the simulated reads were made.

# Build database with all influenza sequences from NCBI
In addition to the NCBI RefSeq sequences, this version of the database will contain all influenza genomes from NCBI.  



## Download data from Influenza Virus Resource (NCBI)
Tried using the query tool of the [Influenza Virus Resoiurce](https://www.ncbi.nlm.nih.gov/genomes/FLU/Database/nph-select.cgi?go=database) with the following settings:


    - Sequence Type: Nucleotide
    - Type: any
    - Host: Human
    - Country: any
    - Subtype: any
    - Full length only: yes
    - Full length plus: no
    - Date: any

    - Advanced filter:
        - include pandemic (H1N1) viruses ([details](https://www.ncbi.nlm.nih.gov/genome/viruses/variation/help/flu-help-center/how-to-use/#pandemic-viruses))
        - include the FLU project
        - exclude lab strains
        - include vaccine strains
        - include lineage defining strains
include mixed subtype

But download keeps failing via the website (times out). This even happens when reducing to just H1N1 suybtype

***

This does not seem to be useable


## Download all influenza from NCBI FTP
Download the full Influenza file from [NCBI FTP site](https://ftp.ncbi.nih.gov/genomes/INFLUENZA/)

In [8]:
export BASE_DIR=/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu/
export ALL_INF_LIB=${BASE_DIR}/library_download_all_influenza_ncbi
export TAX_PATH=${BASE_DIR}/taxonomy_download/

In [24]:
mkdir -p ${ALL_INF_LIB}

In [21]:
cd ${ALL_INF_LIB}
wget https://ftp.ncbi.nih.gov/genomes/INFLUENZA/influenza.fna

--2023-12-19 14:54:00--  https://ftp.ncbi.nih.gov/genomes/INFLUENZA/influenza.fna
Resolving wwwcache.sanger.ac.uk (wwwcache.sanger.ac.uk)... 172.30.152.200
Connecting to wwwcache.sanger.ac.uk (wwwcache.sanger.ac.uk)|172.30.152.200|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 1429095618 (1.3G)
Saving to: ‘influenza.fna’

influenza.fna       100%[===================>]   1.33G  39.1MB/s    in 47s     

2023-12-19 14:54:47 (29.2 MB/s) - ‘influenza.fna’ saved [1429095618/1429095618]



## Test: build a database from the NCBI influenza dowbnload
This is to make sure that kraken2 can build a database using the donwloaded NCBI influenza FASTA file and the taxonomy data that was previously downloaded. This version of the DB will not have segments as taxa.

create a direcotry for the DB

In [12]:
export ALL_NCBI_TEST_DB=${BASE_DIR}/testdb-allflu-noseg

In [12]:
mkdir -p ${ALL_NCBI_TEST_DB}

### prime the new DB with viral RefSeq

In [13]:
kraken2-build \
    --download-library viral \
    --db ${ALL_NCBI_TEST_DB}

Step 1/2: Performing rsync file transfer of requested files
Rsync file transfer complete.
Step 2/2: Assigning taxonomic IDs to sequences
Processed 14972 projects (18639 sequences, 549.88 Mbp)... done.
All files processed, cleaning up extra sequence files... done, library complete.
Masking low-complexity regions of downloaded library... done.


### add the all-infleunza FASTA file

In [14]:
kraken2-build \
    --add-to-library ${ALL_INF_LIB}/influenza.fna \
    --db ${ALL_NCBI_TEST_DB}

Masking low-complexity regions of new file... done.
Added "/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//library_download_all_influenza_ncbi/influenza.fna" to library (/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//testdb-allflu-noseg)


### Copy taxonomy download
This was downloaded earlier with krakentools build command, so now we can just copy to the new DB

In [18]:
cp -r ${TAX_PATH}/taxonomy ${ALL_NCBI_TEST_DB}

In [19]:
tree ${ALL_NCBI_TEST_DB}

/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//testdb-allflu-noseg
├── library
│   ├── added
│   │   ├── 0WCBagKwHO.fna
│   │   ├── 0WCBagKwHO.fna.masked
│   │   └── prelim_map_V4K5Ij90bv.txt
│   └── viral
│       ├── assembly_summary.txt
│       ├── library.fna
│       ├── library.fna.masked
│       ├── manifest.txt
│       └── prelim_map.txt
└── taxonomy
    ├── accmap.dlflag
    ├── citations.dmp
    ├── delnodes.dmp
    ├── division.dmp
    ├── gc.prt
    ├── gencode.dmp
    ├── images.dmp
    ├── merged.dmp
    ├── names.dmp
    ├── nodes.dmp
    ├── nucl_gb.accession2taxid
    ├── nucl_wgs.accession2taxid
    ├── readme.txt
    ├── taxdump.dlflag
    ├── taxdump.tar.gz
    └── taxdump.untarflag

4 directories, 24 files


### build the database

In [20]:
kraken2-build \
    --build \
    --db ${ALL_NCBI_TEST_DB}

Creating sequence ID to taxonomy ID map (step 1)...


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Completed processing of 836203 sequences, 1850603812 bp accession IDs...
Writing data to disk...  complete.
Database files completed. 54.801s]
Database construction complete. [Total: 17m14.131s]


### Test with simulated data
Make sure this database works. As before, use the simulated reads from COVID, RSV and influenza viurses to generate a kraken report.  
NOTE: this DB is not to be used in production because it contains incomplete flu genomes which are also not split into segments. 

In [21]:
export TEST_BASE_DIR_2=/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu_test_allncbi_noseg/

In [22]:
mkdir -p ${TEST_BASE_DIR_2}

In [23]:
kraken2 \
    --db ${ALL_NCBI_TEST_DB} \
    --output ${TEST_BASE_DIR_2}/output.kraken \
    --paired \
    --classified-out ${TEST_BASE_DIR_2}/class_seqs#.fq \
    --unclassified-out ${TEST_BASE_DIR_2}/unclass_seqs#.fq \
    --report ${TEST_BASE_DIR_2}/report.txt \
/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/sim_reads/art/mix/mix_of_all_01-1.fq \
/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/sim_reads/art/mix/mix_of_all_01-2.fq

Loading database information... done.
6620 sequences (1.99 Mbp) processed in 0.186s (2135.0 Kseq/m, 640.50 Mbp/m).
  6618 sequences classified (99.97%)
  2 sequences unclassified (0.03%)


In [25]:
cat ${TEST_BASE_DIR_2}/report.txt 

  0.03	2	2	U	0	unclassified
 99.97	6618	0	R	1	root
 99.97	6618	0	D	10239	  Viruses
 99.97	6618	0	D1	2559587	    Riboviria
 99.97	6618	0	K	2732396	      Orthornavirae
 69.91	4628	0	P	2497569	        Negarnaviricota
 39.43	2610	0	P1	2497571	          Polyploviricotina
 39.43	2610	0	C	2497577	            Insthoviricetes
 39.43	2610	0	O	2499411	              Articulavirales
 39.43	2610	0	F	11308	                Orthomyxoviridae
 25.53	1690	0	G	197911	                  Alphainfluenzavirus
 25.53	1690	0	S	2955291	                    Alphainfluenzavirus influenzae
 25.53	1690	1273	S1	11320	                      Influenza A virus
  3.96	262	118	S2	119210	                        H3N2 subtype
  1.37	91	91	S3	335341	                          Influenza A virus (A/New York/392/2004(H3N2))
  0.03	2	2	S3	1978707	                          Influenza A virus (A/Santa Cruz do Sul/LACENRS-913/2011(H3N2))
  0.02	1	1	S3	1517335	                          Influenza A virus (A/Singapore/H2013.384/2013(H3N2))
 

### Conclusions
Combining the kraken2 pre-bult NCBI viral RefSeq data with the complete NCBI influenza genome file in one database has been successful.

The fact that some genomes are present in both, the RefSeq set and the complete influenza genomes, has not caused any issues for kraken2. It seems that such suplication is dealt with at the database building step.

This test shows that the large NCBI influenza genomes file can be used successfully to build a kraken2 database, including the lookup of the taxonomy data, and that it can be combined with the RefSeq data. 

In the next step, we need to apply the kraken_flu tool to the NCBI influenza data and the RefSeq data to split influenza genomes into segments

## Build DB with only NCBI all influenza dataset but split into segments
This is a further test to confirm that a segmented "all influenza" dataset will work. This DB does not yet contain vire RefSeq. In contrast to the above version, this DB has the influenza genomes split into segments.

Name of the database: testdb-allncbiflu-norefseq


In [5]:
# path to the new DB
echo ${BASE_DIR}/testdb-allncbiflu-norefseq

/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//testdb-allncbiflu-norefseq


In [13]:
mkdir -p ${BASE_DIR}/testdb-allncbiflu-norefseq

### Filter the NCBI infuenza genomes
Use the filter function to discard all non-complete gnomes from the NCBI file. Using the ```--discard_duplicates``` option to ignore (and discard) records where the virus name and segment number is not unique. Such cases are (usually) partial sequences of the different genes on the same segment and we are not interested in such partial segments in any case.   

The output file has to be named 'library.fna' as this file is expected by the 2nd step.

In [9]:
~/kraken_flu/bin/kraken_flu filter \
    --in_fasta ${ALL_INF_LIB}/influenza.fna \
    --out_fasta ${ALL_INF_LIB}/library.fna \
    --discard_duplicates

2024-01-15 19:23:56,397 scanning file /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//library_download_all_influenza_ncbi/influenza.fna for FASTA headers with accepted pattern
2024-01-15 19:24:06,528 found 106374 genomes (unique names) before filtering
2024-01-15 19:24:06,713 341264 FASTA records pass the filters and will be written to output file
2024-01-15 19:24:06,795 starting to write filtered genomes to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//library_download_all_influenza_ncbi/library.fna
2024-01-15 19:24:14,823 finished writing filtered genomes to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//library_download_all_influenza_ncbi/library.fna


### Create the segmented genome files
Use the build function of kraken_flu to create the segment genome file.  
The ```library_path``` is the one used above to create the filtere all-NCBI influenza genomes file. The build tool expects a file ```library.fna``` in this folder, which is how the file was named in the above filter command.  

The --acc2taxid parameter is needed in this case because the sequences from the Influenza resource do not have a taxid in the FASTA header and need their taxid looked up in this foile from NCBI.

In [18]:
~/kraken_flu/bin/kraken_flu build \
    --library_path ${ALL_INF_LIB} \
    --taxonomy_path ${TAX_PATH}/taxonomy \
    --acc2tax_path ${TAX_PATH}/taxonomy/nucl_gb.accession2taxid \
    --out_dir ${BASE_DIR}/testdb-allncbiflu-norefseq

2024-01-15 19:37:34,358 writing modified FASTA file to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//testdb-allncbiflu-norefseq/library/library.fna
2024-01-15 19:37:34,368 scanning file /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//library_download_all_influenza_ncbi/library.fna for influenza viruses
2024-01-15 19:37:37,618 done - found 243752 segment sequences in FASTA file
2024-01-15 19:37:37,618 reading NCBI acc2taxid file to assign taxon IDs to NCBI IDs
2024-01-15 19:42:51,637 done reading NCBI acc2taxid file
2024-01-15 19:42:59,186 finished writing modified FASTA file
2024-01-15 19:42:59,186 no prelim_map.txt file provided in inputs - nothing to do
2024-01-15 19:42:59,604 writing modified names file to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//testdb-allncbiflu-norefseq/taxonomy/names.dmp
2024-01-15 19:42:59,906 finished writing modified names file
2024-01-15 19:43:00,290 writing modified nodes

### Add the segmented influenza genome file to DB
Add the modified NCBI influenza FASTA file to the DB (creates some required files)

In [20]:
kraken2-build \
    --add-to-library ${BASE_DIR}/testdb-allncbiflu-norefseq/library/library.fna \
    --db ${BASE_DIR}/testdb-allncbiflu-norefseq

Masking low-complexity regions of new file... done.
Added "/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//testdb-allncbiflu-norefseq/library/library.fna" to library (/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//testdb-allncbiflu-norefseq)


### add seqid2taxid file
Provide as a symlink. Not clear why it is needed. All sequences in this file have a taxid already in the FASTA header but kraken2 insists to have this file

In [23]:
ln -s ${TAX_PATH}/taxonomy/nucl_gb.accession2taxid ${BASE_DIR}/testdb-allncbiflu-norefseq/taxonomy/nucl_gb.accession2taxid

In [24]:
tree ${BASE_DIR}/testdb-allncbiflu-norefseq

/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//testdb-allncbiflu-norefseq
├── accmap_file.tmp
├── library
│   ├── added
│   │   ├── prelim_map.txt
│   │   ├── prelim_map_UAuqUhkrKx.txt
│   │   ├── _u5SU89tL6.fna
│   │   └── _u5SU89tL6.fna.masked
│   └── library.fna
├── seqid2taxid.map.tmp
└── taxonomy
    ├── names.dmp
    ├── nodes.dmp
    ├── nucl_gb.accession2taxid -> /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//taxonomy_download//taxonomy/nucl_gb.accession2taxid
    └── prelim_map.txt

3 directories, 11 files


### build the database

In [25]:
kraken2-build \
    --build \
    --db ${BASE_DIR}/testdb-allncbiflu-norefseq

Creating sequence ID to taxonomy ID map (step 1)...


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Found 97512/97512 targets, searched through 225690352 accession IDs, search complete.
Sequence ID to taxonomy ID map complete. 27.411s]
Estimating required capacity (step 2)...
Estimated hash table requirement: 16550764 bytes
Capacity estimation complete. [33.433s]
Building database files (step 3)...
Taxonomy parsed and converted.
CHT created with 19 bits reserved for taxid.
Completed processing of 682528 sequences, 1146918922 bp
Writing data to disk...  complete.
Database files completed. 26.228s]
Database construction complete. [Total: 4m27.165s]


### Test with simulated data

In [27]:
export TEST_BASE_DIR_ncbinoref=/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu_test_testdb-allncbiflu-norefseq/

In [28]:
mkdir -p ${TEST_BASE_DIR_ncbinoref}

In [29]:
kraken2 \
    --db ${BASE_DIR}/testdb-allncbiflu-norefseq \
    --output ${TEST_BASE_DIR_ncbinoref}/output.kraken \
    --paired \
    --classified-out ${TEST_BASE_DIR_ncbinoref}/class_seqs#.fq \
    --unclassified-out ${TEST_BASE_DIR_ncbinoref}/unclass_seqs#.fq \
    --report ${TEST_BASE_DIR_ncbinoref}/report.txt \
/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/sim_reads/art/mix/mix_of_all_01-1.fq \
/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/sim_reads/art/mix/mix_of_all_01-2.fq

Loading database information... done.
6620 sequences (1.99 Mbp) processed in 0.149s (2665.8 Kseq/m, 799.73 Mbp/m).
  2662 sequences classified (40.21%)
  3958 sequences unclassified (59.79%)


In [30]:
cat ${TEST_BASE_DIR_ncbinoref}/report.txt

 59.79	3958	3958	U	0	unclassified
 40.21	2662	0	R	1	root
 40.21	2662	0	D	10239	  Viruses
 40.21	2662	0	D1	2559587	    Riboviria
 40.21	2662	0	K	2732396	      Orthornavirae
 40.21	2662	0	P	2497569	        Negarnaviricota
 40.21	2662	0	P1	2497571	          Polyploviricotina
 40.21	2662	0	C	2497577	            Insthoviricetes
 40.21	2662	0	O	2499411	              Articulavirales
 40.21	2662	0	F	11308	                Orthomyxoviridae
 26.31	1742	0	G	197911	                  Alphainfluenzavirus
 26.31	1742	0	S	2955291	                    Alphainfluenzavirus influenzae
 26.31	1742	1345	S1	11320	                      Influenza A virus
  3.99	264	126	S2	119210	                        H3N2 subtype
  1.59	105	0	S3	335341	                          Influenza A virus (A/New York/392/2004(H3N2))
  0.42	28	28	S4	3110328	                            gi|71564872|gb|CY002069|Influenza A virus (A/New York/392/2004(H3N2)) segment 3, complete sequence
  0.39	26	26	S4	3110323	                            gi|7

## Build final DB with RefSeq (segmented) plus all NCBI influenza (segmented)
This version of the DB will contain viral RefSeq, processed by the kraken-flu tool to segment the flu genomes and the "all influenza" dataset from the [NCBI FTP site](https://ftp.ncbi.nih.gov/genomes/INFLUENZA/), also processed by the kraken-flu tool into segmented genomes.

The DB name will be "all_viral_db1"

In [1]:
# these were previously exported, just here for convenience
export BASE_DIR=/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu/
export ALL_INF_LIB=${BASE_DIR}/library_download_all_influenza_ncbi
export TAX_PATH=${BASE_DIR}/taxonomy_download/
export LIB_PATH_VIR=${BASE_DIR}/library_download_viral/

In [2]:
# path to the new DB
export ALL_VIRAL_DB1=${BASE_DIR}/all_viral_db1

In [5]:
mkdir -p ${ALL_VIRAL_DB1}

### Use kraken-flu at v1.2.0
Git install with version number 1.2.0

In [16]:
~/kraken_flu/bin/pip install kraken_flu@git+ssh://git@gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu.git@v1.2.0

  Cloning ssh://****@gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu.git (to revision v1.2.0) to /tmp/pip-install-g9uffkxe/kraken-flu_5cc50da630dd49c8a133a774b60bc7df
  Running command git clone --filter=blob:none --quiet 'ssh://****@gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu.git' /tmp/pip-install-g9uffkxe/kraken-flu_5cc50da630dd49c8a133a774b60bc7df
  Running command git checkout -q 6f8bc70b146d074c5c43a2d2a09db65fd4d2fcc0
  Resolved ssh://****@gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu.git to commit 6f8bc70b146d074c5c43a2d2a09db65fd4d2fcc0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kraken_flu: filename=kraken_flu-1.2.0-py2.py3-none-any.whl size=17672 sha256=397685368fd331ed8c58cc7d9f2e5a3c3a89b19413f407f9270dd62059eb0e77
  Stored in directory: /tmp/pip-ephem-wheel-cache-knnv7g1v/wheels/8f/69/99/c3e67cec731769c91202d07057

In [6]:
~/kraken_flu/bin/kraken_flu -v

kraken_flu 1.2.0


make sure kraken2 is loaded

In [7]:
module load kraken2/2.1.2

        Module loaded. For more information run 'module help kraken2/2.1.2'.
        - By default, kraken2 will search for named databases in /data/pam/software/kraken2 and the current working directory. The full path to
        any database can also be used.
        - `kraken2-build` commands will sometimes experience an error suggesting that it is not possible to download a library using rsync. While the option --use-ftp
        may fix this, rsync tends to be faster and more reliable. Try re-running the `kraken2-build` command until the download is successful. Note that `kraken2-build`
        will produce checkpoints during the installation process and will restart at the last successful step.
        - If `kraken2-build build` freezes or hangs indefinitely, using the `--fast-build` option may help. An earlier version of the software (if available) may also
        allow the build step to proceed successfully.
>


### Prime the DB with viral RefSeq
The kraken2-build rsync download is unreliable and sometimes does not work at all (not sure why). It was done already [here](#downloadrefseq), so here we can just reuse the previsouly downloaded and processed files.

Create the segmented version of the flu genomes from the downloaded viral RefSeq

In [8]:
~/kraken_flu/bin/kraken_flu build \
    --library_path ${LIB_PATH_VIR}/library/viral/ \
    --taxonomy_path ${TAX_PATH}/taxonomy \
    --out_dir ${ALL_VIRAL_DB1}

2024-01-17 10:21:25,021 writing modified FASTA file to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//all_viral_db1/library/library.fna
2024-01-17 10:21:25,063 scanning file /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//library_download_viral//library/viral/library.fna for influenza viruses
2024-01-17 10:21:26,914 done - found 40 segment sequences in FASTA file
2024-01-17 10:21:30,436 finished writing modified FASTA file
2024-01-17 10:21:30,439 writing modified prelim_map.txt file to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//all_viral_db1/library/prelim_map.txt
2024-01-17 10:21:30,497 finished writing modified prelim_map.txt file
2024-01-17 10:21:30,943 writing modified names file to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//all_viral_db1/taxonomy/names.dmp
2024-01-17 10:21:31,001 finished writing modified names file
2024-01-17 10:21:31,443 writing modified nodes file 

In [9]:
# snapshot of the DB folder after this step
tree ${ALL_VIRAL_DB1}

/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//all_viral_db1
├── library
│   ├── library.fna
│   └── prelim_map.txt
└── taxonomy
    ├── names.dmp
    └── nodes.dmp

2 directories, 4 files


### add seqid2taxid file
Provide as a symlink. Not clear why it is needed. All sequences in this file have a taxid already in the FASTA header but kraken2 insists to have this file

In [10]:
ln -s ${TAX_PATH}/taxonomy/nucl_gb.accession2taxid ${ALL_VIRAL_DB1}/taxonomy/nucl_gb.accession2taxid

In [11]:
# snapshot of the DB folder after this step
tree ${ALL_VIRAL_DB1}

/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//all_viral_db1
├── library
│   ├── library.fna
│   └── prelim_map.txt
└── taxonomy
    ├── names.dmp
    ├── nodes.dmp
    └── nucl_gb.accession2taxid -> /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//taxonomy_download//taxonomy/nucl_gb.accession2taxid

2 directories, 5 files


### Filter the NCBI infuenza genomes
Use the filter function to discard all non-complete gnomes from the NCBI file. Using the ```--discard_duplicates``` option to ignore (and discard) records where the virus name and segment number is not unique. Such cases are (usually) partial sequences of the different genes on the same segment and we are not interested in such partial segments in any case.   

The output file has to be named 'library.fna' as this file is expected by the 2nd step.

The output file does NOT need to be in the new DB folder. It is just an intermediate step. Creating it in the download folder so it can be re-used.

Note: this was run previously on the same file and could have just re-used that output. Just re-running here to have a complete list of commands to build the final DB.

In [12]:
~/kraken_flu/bin/kraken_flu filter \
    --in_fasta ${ALL_INF_LIB}/influenza.fna \
    --out_fasta ${ALL_INF_LIB}/library.fna \
    --discard_duplicates

2024-01-17 10:25:43,135 scanning file /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//library_download_all_influenza_ncbi/influenza.fna for FASTA headers with accepted pattern
2024-01-17 10:25:52,006 found 106374 genomes (unique names) before filtering
2024-01-17 10:25:52,211 341264 FASTA records pass the filters and will be written to output file
2024-01-17 10:25:52,263 starting to write filtered genomes to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//library_download_all_influenza_ncbi/library.fna
2024-01-17 10:26:00,028 finished writing filtered genomes to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//library_download_all_influenza_ncbi/library.fna


### Create the segmented genome files
Use the build function of kraken_flu to create the segment genome file.  
The ```library_path``` is the one used above to create the filtere all-NCBI influenza genomes file. The build tool expects a file ```library.fna``` in this folder, which is how the file was named in the above filter command.  

The --acc2taxid parameter is needed in this case because the sequences from the Influenza resource do not have a taxid in the FASTA header and need their taxid looked up in this foile from NCBI.

Path ``--taxonomy_path``` points to the taxonomy directory in the new database folder where the names.dmp and nodes.dmp files have already been modified by the kraken-flu process on the RefSeq dataset. The effect of this is that the additinla names/nodes from the "all influenza" dataset will be added on top of those modifications, which is essnetial in order to keep all modifications and not overwrite them.   
__TODO__: kraken-flu should have a way of handling multiple datasets in one go.

Output to a temporary directory. The kraken2-build tool needs to be run on these files, so they will not be added to the final DB directly. In addition, the current behaviour of the kraken_flu tool would overwrite the library.fna FASTA file from the above viral RefSeq import. This will be changed in a future version

In [14]:
mkdir -p ${BASE_DIR}/tmp/all_viral_tmp/

In [15]:
~/kraken_flu/bin/kraken_flu build \
    --library_path ${ALL_INF_LIB} \
    --taxonomy_path ${ALL_VIRAL_DB1}/taxonomy \
    --acc2tax_path ${ALL_VIRAL_DB1}/taxonomy/nucl_gb.accession2taxid \
    --out_dir ${BASE_DIR}/tmp/all_viral_tmp/

2024-01-17 10:35:12,410 writing modified FASTA file to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//tmp/all_viral_tmp/library/library.fna
2024-01-17 10:35:12,422 scanning file /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//library_download_all_influenza_ncbi/library.fna for influenza viruses
2024-01-17 10:35:15,742 done - found 254776 segment sequences in FASTA file
2024-01-17 10:35:15,742 reading NCBI acc2taxid file to assign taxon IDs to NCBI IDs
2024-01-17 10:38:58,326 done reading NCBI acc2taxid file
2024-01-17 10:39:05,597 finished writing modified FASTA file
2024-01-17 10:39:05,597 no prelim_map.txt file provided in inputs - nothing to do
2024-01-17 10:39:06,043 writing modified names file to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//tmp/all_viral_tmp/taxonomy/names.dmp
2024-01-17 10:39:06,261 finished writing modified names file
2024-01-17 10:39:06,751 writing modified nodes file to /lustre/s

In [16]:
# snapshot of the temp directory that was created in the above command
tree ${BASE_DIR}/tmp/all_viral_tmp/

/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//tmp/all_viral_tmp/
├── library
│   └── library.fna
└── taxonomy
    ├── names.dmp
    └── nodes.dmp

2 directories, 3 files


demonstrate that the taxonomy files in this temp dir contain the viral RefSeq as well as the additional "all influenza" segment data. Looked up the last entry in the viral RefSeq modified names.dmp file and search for it in the tmp dir, adding the next 10 lines after that. It shows that both the RefSeq and the new "all influenza" added names are present

In [17]:
grep -A 10 NC_026432.1 ${BASE_DIR}/tmp/all_viral_tmp/taxonomy/names.dmp

3108310	|	NC_026432.1 Influenza A virus (A/California/07/2009(H1N1)) segment 8 nuclear export protein (NEP) and nonstructural protein 1 (NS1) genes, complete cds	|		|	scientific name	|
3108311	|	gi|290747|gb|L07370|Influenza A virus (A/Memphis/8/1988(H3N2)) segment 5 nucleoprotein (NP) gene, complete cds	|		|	scientific name	|
3108312	|	gi|324507|gb|J02146|Influenza A virus (A/Puerto Rico/8/1934(H1N1)) segment 6, complete sequence	|		|	scientific name	|
3108313	|	gi|324709|gb|J02147|Influenza A virus (A/Puerto Rico/8/1934(H1N1)) segment 5, complete sequence	|		|	scientific name	|
3108314	|	gi|324833|gb|J02150|Influenza A virus (A/Puerto Rico/8/34(H1N1)) segment 8, complete sequence	|		|	scientific name	|
3108315	|	gi|325056|gb|M22579|Influenza A virus (A/swine/Germany/2/1981(H1N1)) segment 5 nucleoprotein gene, complete cds	|		|	scientific name	|
3108316	|	gi|21693162|gb|AF389115|Influenza A virus (A/Puerto Rico/8/34/Mount Sinai(H1N1)) segment 1, complete sequence	|		|	scientific name	

### Combine modified taxonomy files
___TODO___: This needs to become a step that is handled by the kraken_flu tool

Both datsets (RefSeq and "all influenza" have been processed by the kraken_flu tool to treat flu genome segments as taxa in their own rights, which is the whole point of the kraken-flu tool.  
This results in modified names.dmp and nodes.dmp files which now need to be copied back into the DB taxonomy folder, which currently contains the modified files from the RefSeq dataset only.   
Since the taxonomy files in the tmp folder have been built on top of the RefSeq modified taxonomy files, we can simply overwrite the current files with the versions in the tmp dir.

Taxonomy files (names and nodes): overwrite the existing files with the files from the tmp dir. As demonstrated above, the tmp dir files are a combination of the viral RefSeq and the "all influenza" dataset.

In [18]:
cp  ${BASE_DIR}/tmp/all_viral_tmp/taxonomy/n*.dmp ${ALL_VIRAL_DB1}/taxonomy/

In [20]:
# double-checking that the file in the final DB folder contains modifications from both datasets (see above)
grep -A 10 NC_026432.1 ${ALL_VIRAL_DB1}/taxonomy/names.dmp

3108310	|	NC_026432.1 Influenza A virus (A/California/07/2009(H1N1)) segment 8 nuclear export protein (NEP) and nonstructural protein 1 (NS1) genes, complete cds	|		|	scientific name	|
3108311	|	gi|290747|gb|L07370|Influenza A virus (A/Memphis/8/1988(H3N2)) segment 5 nucleoprotein (NP) gene, complete cds	|		|	scientific name	|
3108312	|	gi|324507|gb|J02146|Influenza A virus (A/Puerto Rico/8/1934(H1N1)) segment 6, complete sequence	|		|	scientific name	|
3108313	|	gi|324709|gb|J02147|Influenza A virus (A/Puerto Rico/8/1934(H1N1)) segment 5, complete sequence	|		|	scientific name	|
3108314	|	gi|324833|gb|J02150|Influenza A virus (A/Puerto Rico/8/34(H1N1)) segment 8, complete sequence	|		|	scientific name	|
3108315	|	gi|325056|gb|M22579|Influenza A virus (A/swine/Germany/2/1981(H1N1)) segment 5 nucleoprotein gene, complete cds	|		|	scientific name	|
3108316	|	gi|21693162|gb|AF389115|Influenza A virus (A/Puerto Rico/8/34/Mount Sinai(H1N1)) segment 1, complete sequence	|		|	scientific name	

### use kraken2 build tool to add "all influenza" set to DB
Not entirely sure why this step is needed. Previously tried to simply append the FASTA file from the "all influenza" tmp dir to the already existing library.fna in the DB dir. The expectaion was that this would provide all the data needed to build the final database but the result only contained the RefSeq genomes and none of the "all influenza" (result of running kraken2 with simualted data and also running kraken2-insepct on the final DB). 

In [21]:
kraken2-build \
    --add-to-library ${BASE_DIR}/tmp/all_viral_tmp/library/library.fna \
    --db ${ALL_VIRAL_DB1}

Masking low-complexity regions of new file... done.
Added "/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//tmp/all_viral_tmp/library/library.fna" to library (/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//all_viral_db1)


In [23]:
# snapshot of the DB folder after this step
tree ${ALL_VIRAL_DB1}

/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//all_viral_db1
├── library
│   ├── added
│   │   ├── cZ6g3o2lK2.fna
│   │   ├── cZ6g3o2lK2.fna.masked
│   │   └── prelim_map_jSmcIeIXIr.txt
│   ├── library.fna
│   └── prelim_map.txt
└── taxonomy
    ├── names.dmp
    ├── nodes.dmp
    └── nucl_gb.accession2taxid -> /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//taxonomy_download//taxonomy/nucl_gb.accession2taxid

3 directories, 8 files


### build the final database


In [24]:
kraken2-build \
    --build \
    --db ${ALL_VIRAL_DB1}

Creating sequence ID to taxonomy ID map (step 1)...


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Found 86488/86488 targets, searched through 225690352 accession IDs, search complete.
Sequence ID to taxonomy ID map complete. [43.284s]
Estimating required capacity (step 2)...
Estimated hash table requirement: 664979748 bytes
Capacity estimation complete. [30.992s]
Building database files (step 3)...
Taxonomy parsed and converted.
CHT created with 19 bits reserved for taxid.
Completed processing of 359903 sequences, 1123340241 bp
Writing data to disk...  complete.
Database files completed. 39.035s]
Database construction complete. [Total: 3m53.385s]


In [25]:
# snapshot of the DB folder after this step
tree ${ALL_VIRAL_DB1}

/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//all_viral_db1
├── hash.k2d
├── library
│   ├── added
│   │   ├── cZ6g3o2lK2.fna
│   │   ├── cZ6g3o2lK2.fna.masked
│   │   ├── prelim_map_jSmcIeIXIr.txt
│   │   └── prelim_map.txt
│   ├── library.fna
│   └── prelim_map.txt
├── opts.k2d
├── seqid2taxid.map
├── taxo.k2d
└── taxonomy
    ├── names.dmp
    ├── nodes.dmp
    ├── nucl_gb.accession2taxid -> /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//taxonomy_download//taxonomy/nucl_gb.accession2taxid
    └── prelim_map.txt

3 directories, 14 files


### Inspect the final DB
Make sure all genomes are present and that influenza genomes have been segmented as intended.

run kraken2 inspect tool and save the result in a tmp file

In [26]:
kraken2-inspect --db ${ALL_VIRAL_DB1} > ~/tmp/inspect_all_viral_db1

does the DB contain segmented influenxa genomes from RefSeq (which have an NC_xxxx accesion ID)?

In [28]:
grep "segment 2" ~/tmp/inspect_all_viral_db1 | grep NC_

  0.00	33	33	S4	3108296	                            NC_002021.1 Influenza A virus (A/Puerto Rico/8/1934(H1N1)) segment 2, complete sequence
  0.00	53	53	S3	3108287	                          NC_002205.1 Influenza B virus (B/Lee/1940) segment 2, complete sequence


-> correct

does it also contain non-RefSeq (accession not NC_xxxx)?

In [33]:
grep "segment 2" ~/tmp/inspect_all_viral_db1 | grep -v NC_ | head -5

  0.00	157	157	S4	3166347	                            gi|482578082|gb|KC859228|Influenza A virus (A/swine/Thailand/CB001/2009(H1N1)) segment 2 polymerase PB1 (PB1) gene, complete cds
  0.00	168	168	S4	3169851	                            gi|563408516|gb|KF840458|Influenza A virus (A/swine/England/117316/1986(H1N1)) segment 2 polymerase PB1 (PB1) and putative PB1-F2 protein (PB1-F2) genes, complete cds
  0.00	107	107	S4	3170338	                            gi|584462324|gb|KJ162035|Influenza A virus (A/swine/Thailand/CU-S3350N/2012(H1N1)) segment 2 polymerase PB1 (PB1) and PB1-F2 protein (PB1-F2) genes, complete cds
  0.00	22	22	S4	3202569	                            gi|1017034793|gb|KU976691|Influenza A virus (A/swine/Mexico/AVX30/2012(H1N1)) segment 2 polymerase PB1 (PB1) gene, complete cds; and PB1-F2 pseudogene, complete sequence
  0.00	176	176	S4	3181659	                            gi|827498483|gb|KR700677|Influenza A virus (A/swine/England/1251/2011(H1N1)) segment 2 polymerase PB1 (P

-> correct (not sure why the broken pipe error, cmd works fine in terminal)

### Test with simulated data

In [34]:
export TEST_BASE_DIR_final=/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu_test_final/

In [36]:
mkdir -p ${TEST_BASE_DIR_final}

In [37]:
kraken2 \
    --db ${ALL_VIRAL_DB1} \
    --output ${TEST_BASE_DIR_final}/output.kraken \
    --paired \
    --classified-out ${TEST_BASE_DIR_final}/class_seqs#.fq \
    --unclassified-out ${TEST_BASE_DIR_final}/unclass_seqs#.fq \
    --report ${TEST_BASE_DIR_final}/report.txt \
/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/sim_reads/art/mix/mix_of_all_01-1.fq \
/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/sim_reads/art/mix/mix_of_all_01-2.fq

Loading database information... done.
6620 sequences (1.99 Mbp) processed in 0.180s (2202.9 Kseq/m, 660.88 Mbp/m).
  6618 sequences classified (99.97%)
  2 sequences unclassified (0.03%)


In [38]:
cat ${TEST_BASE_DIR_final}/report.txt 

  0.03	2	2	U	0	unclassified
 99.97	6618	0	R	1	root
 99.97	6618	0	D	10239	  Viruses
 99.97	6618	0	D1	2559587	    Riboviria
 99.97	6618	0	K	2732396	      Orthornavirae
 69.91	4628	0	P	2497569	        Negarnaviricota
 39.43	2610	0	P1	2497571	          Polyploviricotina
 39.43	2610	0	C	2497577	            Insthoviricetes
 39.43	2610	0	O	2499411	              Articulavirales
 39.43	2610	0	F	11308	                Orthomyxoviridae
 25.53	1690	0	G	197911	                  Alphainfluenzavirus
 25.53	1690	0	S	2955291	                    Alphainfluenzavirus influenzae
 25.53	1690	1285	S1	11320	                      Influenza A virus
  4.27	283	137	S2	119210	                        H3N2 subtype
  1.59	105	105	S3	335341	                          Influenza A virus (A/New York/392/2004(H3N2))
  0.14	9	0	S3	1910719	                          Influenza A virus (A/Rhode Island/29/2016(H3N2))
  0.14	9	9	S4	3230699	                            gi|1095416191|gb|KY003383|Influenza A virus (A/Rhode Island/29/2

### Test with real samples from RVI pilot plate 3
Using pre-processed read files from the PAM pipeline in this directory
```/lustre/scratch127/pam/projects/rvidata/bait_capture/rvi-pilot-phase3```

Using this query to identify suitable samples in MLWH
```SQL
SELECT  
	ipm.id_run, 
	ipm.position, 
	ipm.tag_index, 
	CONCAT(ipm.id_run, '_', ipm.position, '#', ipm.tag_index) AS sample,
	supplier_name,
	REGEXP_REPLACE(supplier_name, '^.*X_', '') AS presumed_orig_sample_name
	
FROM iseq_product_metrics ipm 
JOIN iseq_flowcell USING(id_iseq_flowcell_tmp)
JOIN sample USING(id_sample_tmp) 
WHERE ipm.id_run=48106

order by presumed_orig_sample_name
```


#### Test with a flu 100k copies control sample
sample suppplier name: SPRI_1X_Flu_H3N2_100K   
lanelet: 48106_2#10
trimmed reads:
```
/lustre/scratch127/pam/projects/rvidata/bait_capture/rvi-pilot-phase3/results_minreadlen100bp/48106_2#10/trimmed_reads/48106_2#10_1_kneaddata_paired_1.fastq.gz
/lustre/scratch127/pam/projects/rvidata/bait_capture/rvi-pilot-phase3/results_minreadlen100bp/48106_2#10/trimmed_reads/48106_2#10_1_kneaddata_paired_2.fastq.gz
```

In [41]:
export TEST_BASE_DIR_final_48106_2_10=/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu_test_final_48106_2_10/

In [42]:
mkdir -p ${TEST_BASE_DIR_final_48106_2_10}

In [43]:
kraken2 \
    --db ${ALL_VIRAL_DB1} \
    --output ${TEST_BASE_DIR_final_48106_2_10}/output.kraken \
    --paired \
    --classified-out ${TEST_BASE_DIR_final_48106_2_10}/class_seqs#.fq \
    --unclassified-out ${TEST_BASE_DIR_final_48106_2_10}/unclass_seqs#.fq \
    --report ${TEST_BASE_DIR_final_48106_2_10}/report.txt \
/lustre/scratch127/pam/projects/rvidata/bait_capture/rvi-pilot-phase3/results_minreadlen100bp/48106_2#10/trimmed_reads/48106_2#10_1_kneaddata_paired_1.fastq.gz \
/lustre/scratch127/pam/projects/rvidata/bait_capture/rvi-pilot-phase3/results_minreadlen100bp/48106_2#10/trimmed_reads/48106_2#10_1_kneaddata_paired_2.fastq.gz

Loading database information... done.
13862 sequences (3.79 Mbp) processed in 0.268s (3101.2 Kseq/m, 848.25 Mbp/m).
  13862 sequences classified (100.00%)
  0 sequences unclassified (0.00%)


In [44]:
cat ${TEST_BASE_DIR_final_48106_2_10}/report.txt

100.00	13862	0	R	1	root
100.00	13862	0	D	10239	  Viruses
100.00	13862	0	D1	2559587	    Riboviria
100.00	13862	0	K	2732396	      Orthornavirae
100.00	13862	0	P	2497569	        Negarnaviricota
100.00	13862	0	P1	2497571	          Polyploviricotina
100.00	13862	0	C	2497577	            Insthoviricetes
100.00	13862	0	O	2499411	              Articulavirales
100.00	13862	0	F	11308	                Orthomyxoviridae
100.00	13862	0	G	197911	                  Alphainfluenzavirus
100.00	13862	0	S	2955291	                    Alphainfluenzavirus influenzae
100.00	13862	9017	S1	11320	                      Influenza A virus
 31.47	4363	2229	S2	119210	                        H3N2 subtype
 10.34	1433	1433	S3	335341	                          Influenza A virus (A/New York/392/2004(H3N2))
  0.79	110	0	S3	1910719	                          Influenza A virus (A/Rhode Island/29/2016(H3N2))
  0.79	110	110	S4	3230699	                            gi|1095416191|gb|KY003383|Influenza A virus (A/Rhode Island/29/2016(H3

### Conclusion
There is a problem with the kraken2 DB, exemplified by these rows from the report:
```
100.00	13862	9017	S1	11320	                      Influenza A virus
 31.47	4363	2229	S2	119210	                        H3N2 subtype
 10.34	1433	1433	S3	335341	                          Influenza A virus (A/New York/392/2004(H3N2))
  0.79	110	0	S3	1910719	                          Influenza A virus (A/Rhode Island/29/2016(H3N2))
  0.79	110	110	S4	3230699	                            gi|1095416191|gb|KY003383|Influenza A virus (A/Rhode Island/29/2016(H3N2)) segment 7 matrix protein 2 (M2) and matrix protein 1 (M1) genes, complete cds
```
A large number of the H3N2 hits have been assigned to A/New York/392/2004(H3N2) (row 3) but there is no read assigned to any segment. This shouid not be possible because all sequences should now be assigned at segment level and thus any hit to a higher-order taxon should have hits to a segment unless all hits assigned to the higher-order taxon are equally matching all 8 segments which seems unlikely (they would all have to be from the panhandles).
Further check of the kraken2-inspect file also reveals that the assignment of new taxa to segments from the RefSeq dataset has not worked.  
As it has been show above that the kraken-flu tool works as intended on a RefSeq dataset alone, the most likely explanation is that the error was introduced by combinbing the datasets.

A new approach will be taken whereby NCBI RefSeq data will be obtained as raw FASTA and combined with the "all influenza" data before running kraken-flu

## Build final DB without using kraken2 pre-built datasets
As discussed above, combingin the RefSeq and the FTP "all influenza" dataset appears to be causing problems still. It is not clear why this is happening because the files created during the process appear to be correct but the act of combining the data is still somehow causing RefSeq and "all influenza" data to be treateed differently, resulting in loss of segmented taxa for RefSeq only.

As an alternative approach, RefSeq will be downloaded directly from NCBI FTP site https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/  
___NOTE___ There is also https://ftp.ncbi.nlm.nih.gov/genomes/refseq/ but this has one folder per genome and cannot be used without a manifest that downloads one genome at a time. Even trying to open the directory in a browser fails, presumably because there are too many folders to display. Similarly, using this resource via FTP on the command line would onyl really make sense once a path to a specific genome is known.

The assumption is that the summary file we are downloading here, is the concatenated FASTA file of all genomes that are available in the refseg genomes release. There is some explanation of the files here but it is not entirely clear:  
https://www.ncbi.nlm.nih.gov/books/NBK50679/#RefSeqFAQ.what_data_are_available_for_ft  




In [45]:
# these were previously exported, just here for convenience
export BASE_DIR=/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu/
export ALL_INF_LIB=${BASE_DIR}/library_download_all_influenza_ncbi
export TAX_PATH=${BASE_DIR}/taxonomy_download/
export LIB_PATH_VIR=${BASE_DIR}/library_download_viral/

In [47]:
mkdir -p ${ALL_VIRAL_DB_2}

### Download NCBI viral RefSeq without kraken2-build
The above databases used a viral RefSeq dataset that was obtained and pre-processed by the kraken2 build tool.  
To avoid having to process this separately from the "all influenza dataset", here we are downloading the raw FASTA file for viral RefSeq from

https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/


In [53]:
cd /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu/library_download_viral_refseq_ftp
wget https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.1.1.genomic.fna.gz
gunzip viral.1.1.genomic.fna.gz


--2024-01-17 18:08:24--  https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.1.1.genomic.fna.gz
Resolving wwwcache.sanger.ac.uk (wwwcache.sanger.ac.uk)... 172.30.152.200
Connecting to wwwcache.sanger.ac.uk (wwwcache.sanger.ac.uk)|172.30.152.200|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 168645964 (161M) [application/x-gzip]
Saving to: ‘viral.1.1.genomic.fna.gz.2’

viral.1.1.genomic.f 100%[===================>] 160.83M  6.73MB/s    in 25s     

2024-01-17 18:08:49 (6.37 MB/s) - ‘viral.1.1.genomic.fna.gz.2’ saved [168645964/168645964]

gzip: viral.1.1.genomic.fna already exists; do you wish to overwrite (y or n)? 


### Filter NCBI all infleunza FASTA file
Apply the kraken-flu filter command to the NCBI FTP download of "all influenza" sequences to obtain a final set of complete genomes only.

In [55]:
~/kraken_flu/bin/kraken_flu filter \
    --in_fasta ${ALL_INF_LIB}/influenza.fna \
    --out_fasta ${ALL_INF_LIB}/whole_genome_influenza.fna \
    --discard_duplicates

2024-01-17 18:16:25,162 scanning file /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//library_download_all_influenza_ncbi/influenza.fna for FASTA headers with accepted pattern
2024-01-17 18:16:39,703 found 106374 genomes (unique names) before filtering
2024-01-17 18:16:40,003 341264 FASTA records pass the filters and will be written to output file
2024-01-17 18:16:40,081 starting to write filtered genomes to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//library_download_all_influenza_ncbi/whole_genome_influenza.fna
2024-01-17 18:16:51,735 finished writing filtered genomes to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//library_download_all_influenza_ncbi/whole_genome_influenza.fna


### Combine FASTA files
Create a combined resource by concatenating the FASTA files for viral RefSeq and the "all influenza" FTP download into a new file

In [57]:
cat /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu/library_download_viral_refseq_ftp/viral.1.1.genomic.fna \
${ALL_INF_LIB}/whole_genome_influenza.fna > \
${BASE_DIR}/tmp/combined_refseq_all_influenza.fna

check numbers

In [58]:
grep -c "^>" /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu/library_download_viral_refseq_ftp/viral.1.1.genomic.fna \
${ALL_INF_LIB}/whole_genome_influenza.fna \
${BASE_DIR}/tmp/combined_refseq_all_influenza.fna

/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu/library_download_viral_refseq_ftp/viral.1.1.genomic.fna:18719
/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//library_download_all_influenza_ncbi/whole_genome_influenza.fna:341264
/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//tmp/combined_refseq_all_influenza.fna:359983


The kraken-flu tool currently expects the FASTA file to be called 'library.fna' as per kraken2 convention.   
Provide a symlink under this name

In [59]:
ln -s ${BASE_DIR}/tmp/combined_refseq_all_influenza.fna ${BASE_DIR}/tmp/library.fna

### Create the segmented genome files
Use the build function of kraken_flu to create the segment genome file.  
The ```library_path``` is the tmp path used above to create the concatenated FASTA file. The kraken-flu tool expects the file to be called 'library.fna' hence the symlink.  

The --acc2taxid parameter is needed in this case because the sequences from both resources do not have kraken2 taxids in the header yet.

Path ``--taxonomy_path``` points to the NCBI download of the taxonomy files that was created earlier.


In [60]:
~/kraken_flu/bin/kraken_flu build \
    --library_path ${BASE_DIR}/tmp/ \
    --taxonomy_path ${TAX_PATH}/taxonomy \
    --acc2tax_path ${TAX_PATH}/taxonomy/nucl_gb.accession2taxid \
    --out_dir ${ALL_VIRAL_DB_2}

2024-01-17 18:33:00,317 writing modified FASTA file to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//all_viral_db_2/library/library.fna
2024-01-17 18:33:00,332 scanning file /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//tmp/library.fna for influenza viruses
2024-01-17 18:33:08,283 done - found 254816 segment sequences in FASTA file
2024-01-17 18:33:08,283 reading NCBI acc2taxid file to assign taxon IDs to NCBI IDs
2024-01-17 18:38:44,576 done reading NCBI acc2taxid file
2024-01-17 18:38:56,645 finished writing modified FASTA file
2024-01-17 18:38:56,645 no prelim_map.txt file provided in inputs - nothing to do
2024-01-17 18:38:57,207 writing modified names file to /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//all_viral_db_2/taxonomy/names.dmp
2024-01-17 18:38:57,544 finished writing modified names file
2024-01-17 18:38:57,996 writing modified nodes file to /lustre/scratch126/gsu/team112/personal/fs5/rvi

### add seqid2taxid file
Provide as a symlink. Not clear why it is needed. All sequences in this file have a taxid already in the FASTA header but kraken2 insists to have this file

In [62]:
ln -s ${TAX_PATH}/taxonomy/nucl_gb.accession2taxid ${ALL_VIRAL_DB_2}/taxonomy/nucl_gb.accession2taxid

In [63]:
tree ${ALL_VIRAL_DB_2}

/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//all_viral_db_2
├── library
│   └── library.fna
└── taxonomy
    ├── names.dmp
    ├── nodes.dmp
    └── nucl_gb.accession2taxid -> /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//taxonomy_download//taxonomy/nucl_gb.accession2taxid

2 directories, 4 files


### use kraken2-build to add the FASTA file to the library

In [64]:
kraken2-build \
    --add-to-library ${ALL_VIRAL_DB_2}/library/library.fna \
    --db ${ALL_VIRAL_DB_2}

Masking low-complexity regions of new file... done.
Added "/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//all_viral_db_2/library/library.fna" to library (/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//all_viral_db_2)


In [65]:
tree ${ALL_VIRAL_DB_2}

/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//all_viral_db_2
├── library
│   ├── added
│   │   ├── 6a3Yssvbdw.fna
│   │   ├── 6a3Yssvbdw.fna.masked
│   │   └── prelim_map_smR_sIcs0L.txt
│   └── library.fna
└── taxonomy
    ├── names.dmp
    ├── nodes.dmp
    └── nucl_gb.accession2taxid -> /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//taxonomy_download//taxonomy/nucl_gb.accession2taxid

3 directories, 7 files


remove the raw library.fna file to be on the safe side. Might not cause a problem but not sure. Move to ~/tmp for now

In [66]:
mv ${ALL_VIRAL_DB_2}/library/library.fna ~/tmp

### Build the final database

In [67]:
kraken2-build \
    --build \
    --db ${ALL_VIRAL_DB_2}

Creating sequence ID to taxonomy ID map (step 1)...


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Found 105167/105167 targets, searched through 231179846 accession IDs, search complete.
Sequence ID to taxonomy ID map complete. 8.721s]
Estimating required capacity (step 2)...
Estimated hash table requirement: 667696272 bytes
Capacity estimation complete. [46.646s]
Building database files (step 3)...
Taxonomy parsed and converted.
CHT created with 19 bits reserved for taxid.
Completed processing of 359982 sequences, 1126088992 bp
Writing data to disk...  complete.
Database files completed. 19.520s]
Database construction complete. [Total: 7m15.024s]


### Test with simulated data

In [68]:
export TEST_BASE_DIR_final_2=/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu_test_final_2/

In [69]:
mkdir -p ${TEST_BASE_DIR_final_2}

In [70]:
kraken2 \
    --db ${ALL_VIRAL_DB_2} \
    --output ${TEST_BASE_DIR_final_2}/output.kraken \
    --paired \
    --classified-out ${TEST_BASE_DIR_final_2}/class_seqs#.fq \
    --unclassified-out ${TEST_BASE_DIR_final_2}/unclass_seqs#.fq \
    --report ${TEST_BASE_DIR_final_2}/report.txt \
/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/sim_reads/art/mix/mix_of_all_01-1.fq \
/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/sim_reads/art/mix/mix_of_all_01-2.fq

Loading database information... done.
6620 sequences (1.99 Mbp) processed in 0.266s (1492.9 Kseq/m, 447.86 Mbp/m).
  6618 sequences classified (99.97%)
  2 sequences unclassified (0.03%)


In [71]:
cat ${TEST_BASE_DIR_final_2}/report.txt 

  0.03	2	2	U	0	unclassified
 99.97	6618	0	R	1	root
 99.97	6618	0	D	10239	  Viruses
 99.97	6618	0	D1	2559587	    Riboviria
 99.97	6618	0	K	2732396	      Orthornavirae
 69.91	4628	0	P	2497569	        Negarnaviricota
 39.43	2610	0	P1	2497571	          Polyploviricotina
 39.43	2610	0	C	2497577	            Insthoviricetes
 39.43	2610	0	O	2499411	              Articulavirales
 39.43	2610	0	F	11308	                Orthomyxoviridae
 25.53	1690	0	G	197911	                  Alphainfluenzavirus
 25.53	1690	0	S	2955291	                    Alphainfluenzavirus influenzae
 25.53	1690	1285	S1	11320	                      Influenza A virus
  4.27	283	137	S2	119210	                        H3N2 subtype
  1.59	105	105	S3	335341	                          Influenza A virus (A/New York/392/2004(H3N2))
  0.14	9	0	S3	1910719	                          Influenza A virus (A/Rhode Island/29/2016(H3N2))
  0.14	9	9	S4	3230699	                            gi|1095416191|gb|KY003383|Influenza A virus (A/Rhode Island/29/2